In [3]:
import pandas as pd
from datetime import date
from gmail_api import init_gmail_service, create_draft_email, send_draft_email

# 1. Setup Gmail Service
# This uses your local 'client_secret.json' and handles authentication
CLIENT_FILE = 'client_secret.json'

service = init_gmail_service(CLIENT_FILE)

gmail v1 service created successfully


In [4]:
def send_invoice_email(service, receiver_email, name, due_date, invoice_no, amount):
    """
    Creates and sends an email using the Gmail API (replacing the SMTP function).
    """
    subject = f"Invoice Reminder: {invoice_no}"
    
    # HTML Body
    # "f" allows you to inject variables inside the string.
    body = f"""
    <html>
      <body>
        <p>Hi {name},</p>
        <p>I hope you are well.</p>
        <p>I just wanted to drop you a quick note to remind you that <strong>{amount} USD</strong> 
           in respect of our invoice {invoice_no} is due for payment on <strong>{due_date}</strong>.</p>
        <p>I would be really grateful if you could confirm that everything is on track for payment.</p>
        <p>Best regards,</p>
        <p>Your Name</p>
      </body>
    </html>
    """

    # Step 1: Create the draft
    draft = create_draft_email(service, to=receiver_email, subject=subject, body=body, body_type='html')
    
    # Step 2: Send the draft immediately
    sent_message = send_draft_email(service, draft['id'])
    return sent_message

def process_invoices(file):
    # Pandas reads everything as strings (text) by default, so we use "parse_dates" so it converts
    # them into Timestamp objects so It will deal with them as dates like: 2025-01-11 → a real calendar date.
    df = pd.read_csv(file, parse_dates=['due_date', 'reminder_date'])
    
    present = date.today() # Gets specifically today's date from your computer's system clock.
    email_counter = 0

    print(f"Checking invoices for date: {present}")

    # this loop returns "_" the index we don't need it, "row" the row data.
    for _, row in df.iterrows():
        # Check if reminder_date is today (or passed) AND not paid
        if (present >= row["reminder_date"].date()) and (row["has_paid"].lower() == "no"):
            
            print(f"Sending email to {row['name']} ({row['email']})...")
            
            send_invoice_email(
                service=service,
                receiver_email=row["email"],
                name=row["name"],
                due_date=row["due_date"].strftime("%d, %b %Y"), # Formats date like: 11, Aug 2022
                invoice_no=row["invoice_no"],
                amount=row["amount"]
            )
            email_counter += 1

    return f"Done! Total Emails Sent: {email_counter}"

In [5]:
# The condition means: Only run the code below if this file is executed directly, not imported.
# If we don't do it that will make anyone that imports the file can read and run the functions.
if __name__ == "__main__":
    result = process_invoices('data.csv')
    print(result)

Checking invoices for date: 2026-01-13
Sending email to Kumar (hamza.jbn123@icloud.com)...
Done! Total Emails Sent: 1
